In [1]:
import pandas as pd
from datetime import timedelta
from scipy.spatial.distance import cdist
import warnings
from pandas.errors import DtypeWarning
from scipy.spatial import cKDTree
import numpy as np
from pykrige.ok import OrdinaryKriging
warnings.simplefilter(action='ignore', category=DtypeWarning)
warnings.filterwarnings('ignore')

In [2]:
def kriging_interpolation(known_coords, known_values, unknown_coords):
    # Extract the latitude and longitude from the known coordinates
    lons = known_coords[:, 0]
    lats = known_coords[:, 1]

    # Create an OrdinaryKriging object
    OK = OrdinaryKriging(
        lons, lats, known_values, variogram_model='linear',
        verbose=False, enable_plotting=False
    )

    # Extract the latitude and longitude for the unknown coordinates
    unknown_lons = unknown_coords[:, 0]
    unknown_lats = unknown_coords[:, 1]
    print(len(unknown_lons))
    print(len(unknown_lats))
    # Perform the Kriging interpolation
    z, ss = OK.execute('grid', unknown_lons, unknown_lats)
    # Return the interpolated values
    return z.flatten()

In [3]:

def perform_kriging_batchwise(group, feature_name, batch_size=100):
    results = []
    if feature_name =='ic':
        group[feature_name] = pd.to_numeric(group[feature_name], errors='coerce')

    known_coords = group[['lon_x', 'lat_x']].values  # Known coordinates (stations)
    known_values = group[feature_name].values        # Known values, using the passed feature name
    known_values = known_values.astype(float)
    
    # Split the unknown coordinates into batches
    unknown_coords = group[['lon_y', 'lat_y']].drop_duplicates().values  # Unknown coordinates (fire points)
    print(unknown_coords)
    total_points = len(unknown_coords)
    
    for i in range(0, total_points, batch_size):
        batch_coords = unknown_coords[i:i+batch_size]
        n = len(batch_coords)
        interpolated_values = kriging_interpolation(known_coords, known_values, batch_coords)  # Perform Kriging interpolation
        # print(batch_coords)
        # print(interpolated_values)
        # # interpolated_values = interpolated_values[:n]
        # print("Length of batch_coords:", len(batch_coords))
        # print("Length of interpolated_values:", len(interpolated_values))

        batch_result = pd.DataFrame({  # Return interpolation result
            'date': group['date'].iloc[0],
            'year': group['year_y'].iloc[0],
            'month': group['month_y'].iloc[0],
            'day': group['day_y'].iloc[0],
            'area': group['area'].iloc[0],
            'lon': batch_coords[:, 0],
            'lat': batch_coords[:, 1],
            'fire': group['fire'].iloc[0],
            feature_name: interpolated_values  # Use feature name as the column name
        })
        results.append(batch_result)
    
    return pd.concat(results, ignore_index=True)


火点数据集处理
---

In [4]:
fire = pd.read_excel('fire2010-2017.xls')
fire['fire'] = 1
# 时间序列扩充
new_rows_list = []

# 遍历所有唯一的火灾日期
for fire_date in fire['图像日期'].unique():
    fire_rows = fire[fire['图像日期'] == fire_date]
    
    # 检查前两天和后一天的窗口
    for offset in range(-10, 10):
        new_fire_date = pd.Timestamp(fire_date) + timedelta(days=offset)
        
        # 检查新日期是否已存在数据
        existing_rows = fire[fire['图像日期'] == new_fire_date]
        
        if existing_rows.empty:  # 如果不存在数据，则创建新行
            new_fire_rows = fire_rows.copy()
            new_fire_rows['图像日期'] = new_fire_date
            new_fire_rows['fire'] = 0
            new_rows_list.append(new_fire_rows)
        else:  # 如果存在数据，则使用原有数据
            new_rows_list.append(existing_rows)

# 使用pandas.concat合并所有新行
new_rows = pd.concat(new_rows_list, ignore_index=True)

extended_fire_data = pd.concat([fire, new_rows], ignore_index=True).drop_duplicates()
# extended_fire_data.to_csv(r'E:\dataset\fire2010-2017_new.csv')

In [5]:
provinces = ['辽宁省', '黑龙江省', '吉林省']
fire = extended_fire_data[extended_fire_data['地区'].isin(provinces)].copy()

name_mapping = {
    '地区': 'area',
    '图像日期': 'time',
    '东经': 'lon',
    '北纬': 'lat',
    'fire': 'fire'
}
fire.rename(columns=name_mapping, inplace=True)

# 转换时间列并添加时间特征
fire['time'] = pd.to_datetime(fire['time'])
fire['year'] = fire['time'].dt.year
fire['month'] = fire['time'].dt.month
fire['day'] = fire['time'].dt.day
fire['hour'] = fire['time'].dt.hour
fire['date'] = fire['time'].dt.date
fire['date'] = pd.to_datetime(fire['date'])

In [6]:
fire.head()

,area,time,lon,lat,fire,year,month,day,hour,date
2164,辽宁省,2010-03-10 12:41:00,120.1782,40.9771,1,2010,3,10,12,2010-03-10
2833,辽宁省,2010-03-30 10:18:00,122.3467,40.4534,1,2010,3,30,10,2010-03-30
2834,辽宁省,2010-03-30 11:23:00,122.3741,40.4504,1,2010,3,30,11,2010-03-30
2837,辽宁省,2010-04-01 13:07:00,121.3900,41.4604,1,2010,4,1,13,2010-04-01
2838,辽宁省,2010-04-01 13:49:00,121.3935,41.4570,1,2010,4,1,13,2010-04-01


站点数据读取
---

In [7]:
sta = pd.read_csv('dataset.csv')
sta['date'] = pd.to_datetime(sta[['year', 'month', 'day']])

时间匹配

In [8]:
merged_data = pd.merge(sta, fire, left_on='date', right_on='date', how='inner')
merged_data['distance'] = np.sqrt((merged_data['lon_x'] - merged_data['lon_y'])**2 + 
                                  (merged_data['lat_x'] - merged_data['lat_y'])**2)

print(merged_data)

            sta   Alti  year_x  month_x  day_x  TEM_Max  TEM_Min  RHU_Min  \
0         50136  433.0    2010        2     28    -13.0    -40.8       23   
1         50246  361.9    2010        2     28    -12.9    -37.0       27   
2         50247  514.5    2010        2     28    -12.8    -38.5       24   
3         50349  494.6    2010        2     28    -12.3    -33.0       27   
4         50353  177.4    2010        2     28    -13.1    -28.7       30   
...         ...    ...     ...      ...    ...      ...      ...      ...   
38453077  59951   39.9    2017        7      3     30.8     25.3       73   
38453078  59954   35.2    2017        7      3     31.4     25.9       68   
38453079  59954   35.2    2017        7      3     31.4     25.9       68   
38453080  59954   35.2    2017        7      3     31.4     25.9       68   
38453081  59954   35.2    2017        7      3     31.4     25.9       68   

          PRE_Time_2020  Snow_Depth  ...  area                time     lon_

In [15]:
merged_data = merged_data.sort_values(by=['date','area', 'distance']).drop_duplicates(subset='date', keep='first')
# merged_data = merged_data.drop('distance', axis=1, inplace=True)

In [16]:


print(merged_data)
# merged_data.head(100).to_csv('100.csv')

            sta   Alti  year_x  month_x  day_x  TEM_Max  TEM_Min  RHU_Min  \
647       54325  183.8    2010        2     28     -1.3     -6.9       16   
2882      54325  183.8    2010        3      1     -0.1     -9.9       14   
5117      54325  183.8    2010        3      2      6.1     -7.0       14   
7352      54325  183.8    2010        3      3      6.9    -11.9       20   
9587      54325  183.8    2010        3      4      8.7      1.4       22   
...         ...    ...     ...      ...    ...      ...      ...      ...   
38372714  50247  514.5    2017        6     29     26.6     15.0       46   
38390578  50247  514.5    2017        6     30     25.5     13.2       52   
38408442  50247  514.5    2017        7      1     28.9     11.5       37   
38426306  50247  514.5    2017        7      2     32.5     11.6       16   
38444158  50247  514.5    2017        7      3     34.0      9.5       20   

          PRE_Time_2020  Snow_Depth  ...  area                time     lon_

插值到火点（kriging）

In [ ]:
#要插值的要素
# 'Alti', 'TEM_Max', 'TEM_Min', 
#                 'RHU_Min', 'PRE_Time_2020', 'Snow_Depth', 'WIN_S_Max',  
#                 'yth1', 'yth10', 'yth100', 'yth1000', 'kb', 'erc', 'sc', 'bi', 'p', 
#                 'FFMC', 'DMC', 'DC', 'FWI', 'ISI', 'BUI', 'DSR', 'FFDI',
column_names_list = merged_data.columns.tolist()
print(column_names_list)
column_names = [ 'Alti', 'TEM_Max', 'TEM_Min', 
                'RHU_Min', 'PRE_Time_2020', 'Snow_Depth', 'WIN_S_Max',  
                'yth1', 'yth10', 'yth100', 'yth1000', 'kb', 'erc', 'sc', 'bi', 'p', 
                'FFMC', 'DMC', 'DC', 'FWI', 'ISI', 'BUI', 'DSR', 'FFDI','ic'
               
                ]

# 初始化最终插值结果的DataFrame
final_interpolated_results = None

# 循环遍历每个要素，并执行插值
for column_name in column_names:
    # 对当前要素执行插值
    print(merged_data.groupby('date').size())
    interpolated_results = merged_data.groupby('date').apply(lambda group: perform_kriging_batchwise(group, column_name))
    # 重置索引
    interpolated_results.reset_index(drop=True, inplace=True)
    
    # 将插值结果合并到最终结果DataFrame
    if final_interpolated_results is None:
        final_interpolated_results = interpolated_results
    else:
        final_interpolated_results = final_interpolated_results.merge(interpolated_results, on=['date', 'lon', 'lat','year',
                                                                                               'month','day','area','fire'])
final_interpolated_results.to_csv('fire_point_data.csv')

['sta', 'Alti', 'year_x', 'month_x', 'day_x', 'TEM_Max', 'TEM_Min', 'RHU_Min', 'PRE_Time_2020', 'Snow_Depth', 'WIN_S_Max', 'lon_x', 'lat_x', 'yth1', 'yth10', 'yth100', 'yth1000', 'kb', 'erc', 'sc', 'bi', 'ic', 'p', 'FFMC', 'DMC', 'DC', 'FWI', 'ISI', 'BUI', 'DSR', 'FFDI', 'date', 'area', 'time', 'lon_y', 'lat_y', 'fire', 'year_y', 'month_y', 'day_y', 'hour', 'distance']
date
2010-02-28    1
2010-03-01    1
2010-03-02    1
2010-03-03    1
2010-03-04    1
             ..
2017-06-29    1
2017-06-30    1
2017-07-01    1
2017-07-02    1
2017-07-03    1
Length: 979, dtype: int64
[[120.1782  40.9771]]
